# Code for Cross-Domain Evaluation of LLMs

## Install necessary libraries

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install pandas
!pip install torch
!pip install requests
!pip install trl
!pip install datasets
!pip install transformers

## Import necessary libraries

In [ ]:
import re
import os
import torch
import requests
import pandas as pd
from trl import SFTTrainer
from datetime import datetime
from google.colab import drive
from datasets import Dataset, load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments, EarlyStoppingCallback

In [ ]:
# Mount Google Drive to access files and save results

drive.mount('/content/drive', force_remount=True)

In [ ]:
############ UNCOMMENT THE CODE BELOW TO ENABLE TELEGRAM NOTIFICATIONS #############

# api_token = ""  # Provide your Telegram bot API token here
# chat_id = ""  # Provide the chat ID where notifications should be sent

In [ ]:
############ UNCOMMENT AND USE THE FUNCTION BELOW TO ENABLE TELEGRAM NOTIFICATIONS #############

# def notify(text='Cell execution completed.'):
#     requests.post('https://api.telegram.org/' + 'bot{}/sendMessage'.format(api_token), params=dict(chat_id=chat_id, text=text))

## Load Fine-Tuned Model

In [ ]:
model_name = "path/to/fine_tuned_model"  # Specify the path to your fine-tuned model

In [ ]:
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=1024,
        dtype=None,
        load_in_4bit=True,
    )
    print(f"Model: {model_name} and tokenizer loaded successfully.")
    # Uncomment the line below to enable Telegram notifications when the model and tokenizer are loaded
    # notify(f"Model: {model_name} and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error occurred while loading the model or tokenizer: {e}")
    # Uncomment the line below to enable Telegram notifications for errors during model or tokenizer loading
    # notify(f"Error occurred while loading the model or tokenizer: {e}")

## Load Dataset

In [ ]:
def load_csv_to_dataset(file_path):
    df = pd.read_csv(file_path)
    dataset = pd.DataFrame(df)
    return dataset

In [ ]:
test_dataset = load_csv_to_dataset(f"../datasets/IEEE-Rel-3k-Test.csv")

##### The function `analyse_dataset` provides insights into a dataset by displaying its shape, information, descriptive statistics, and category counts for the `simplified_label` column.

In [ ]:
def analyse_dataset(dataset, name):
    print(f"{name} Dataset Analysis:\n")
    df = pd.DataFrame(dataset)
    print(f"Shape: {df.shape}")
    df.info()
    print(f"\nDescriptive Statistics:\n", df.describe(include='all'))
    print(f"\n'simplified_label' Category Counts:\n", df['simplified_label'].value_counts())
    print("-" * 50)

In [ ]:
analyse_dataset(test_dataset, "Test")

## Define Prompt

##### To ensure consistency and fairness in comparison, the same `prompt_template` is used across all experiments, including fine-tuning and cross-domain evaluation.

In [ ]:
prompt_template = '''
Classify the relationship between '[TOPIC-A]' and '[TOPIC-B]'
'''

In [ ]:
prompt_in_use = prompt_template
print(prompt_in_use)

##### The function `test_create_conversation` generates a conversation prompt for each row in the dataset by replacing placeholders (`[TOPIC-A]` and `[TOPIC-B]`) in the `prompt_in_use` template with the corresponding subject and object values.

In [ ]:
def test_create_conversation(row):
    sub = row['subject']
    obj = row['object']

    human = prompt_in_use.replace("[TOPIC-A]", sub).replace("[TOPIC-B]", obj)

    return [{"from": "human", "value": human}]

In [ ]:
# Generate a 'conversations' column in the test dataset by applying the `test_create_conversation` function to each row

test_dataset['conversations'] = test_dataset.apply(test_create_conversation, axis=1)

##### The function `print_first_conversation` displays the first conversation from the `conversations` column of the dataset.

In [ ]:
def print_first_conversation():

    for dataset_name, dataset in zip(["test"], [test_dataset]):
        try:
            first_conversation = dataset['conversations'][0]
            print(f"{dataset_name} dataset:")
            print(first_conversation)
        except (IndexError, KeyError) as e:
            print(f"Error accessing 'conversations' in {dataset_name} dataset: {e}")

In [ ]:
print_first_conversation()

##### The function `formatting_prompts_func` applies a chat template to format the conversations in the dataset.

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

## Data Preparation

In [ ]:
test_dataset = Dataset.from_pandas(test_dataset)

test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

##### The function `mapped_data_analysis` analyses the mapped dataset by displaying its structure, descriptive statistics, and category counts.

In [ ]:
def mapped_data_analysis(dataset, name):
    print(f"{name} Dataset Analysis:\n")
    df = dataset.to_pandas()
    df.info()
    print(f"\nDescriptive Statistics:\n", df.describe(include='all'))
    print(f"\n'simplified_label' Category Counts:\n", df['simplified_label'].value_counts())
    print("-" * 50)

In [ ]:
mapped_data_analysis(test_dataset, "Test")

In [ ]:
# Convert the test dataset to a pandas DataFrame
test_dataset_converted = test_dataset.to_pandas()

test_dataset_converted

In [ ]:
prompt = test_dataset_converted['text'].loc[len(test_dataset)-1]

prompt

## Model Inference

In [ ]:
# Prepares the model for inference by setting it to inference mode
FastLanguageModel.for_inference(model);

In [ ]:
# Tokenize the input prompt and convert it into a format suitable for the model.
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
# Generate test output from the model using the tokenized input.
outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)

# Decode the generated tokens into a human-readable string, skipping special tokens.
text_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
text_output

##### The function `extract_relationship_label` identifies and extracts the relationship label from the output returned by model.

In [ ]:
def extract_relationship_label(text):
  if prompt_in_use == prompt_template:
    words = ["broader", "narrower", "same-as", "other"]

    for word in words:
      if re.search(r"\b" + re.escape(word) + r"\b", text, re.IGNORECASE):
        return word

  else: return "Relation not found!"

In [ ]:
relationship_label = extract_relationship_label(text_output)
print(relationship_label)

##### The function `get_relationship` performs inference on a single data point, extracting the complete output and predicted relationship label.

In [ ]:
def get_relationship(data_point,model,tokenizer):

    FastLanguageModel.for_inference(model)

    inputs = tokenizer(data_point['text'], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    text_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    data_point['complete_output'] = text_output
    data_point['predicted_label'] = extract_relationship_label(text_output)

    return data_point

In [ ]:
# Apply the `get_relationship` function to each row of the `test_dataset_converted` DataFrame.
test_dataset_converted = test_dataset_converted.apply(lambda row: get_relationship(row, model, tokenizer), axis=1)

print(f"Inference for {model_name} is completed.")

# Uncomment the line below to send a notification via Telegram when the inference process is completed.
# notify(f"Inference for {model_name} is completed.") 

In [ ]:
test_dataset_converted

## Save Output

In [ ]:
run_type = "IEEE-Rel-3K"

In [ ]:
file_name = f"{run_type}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
print(file_name)

In [ ]:
test_dataset_converted.to_csv(f'../results/{file_name}.csv', index=False)